In [1]:
import sys
import os
import random
import json

# 1. Add the parent directory (Finetuning) to sys.path to import utils_ft
# This assumes the notebook is running in Finetuning/notebooks/
sys.path.append('..')

from utils_ft import (
    get_benchmark_data_path, 
    get_processed_data_path, 
    get_scripts_path,
    to_relative
)

# 2. Add scripts folder to path so we can import the converter
sys.path.append(get_scripts_path())

from data_converter import main as run_converter

# Define Paths using the Utils
data_in = get_benchmark_data_path()
data_out = get_processed_data_path()

print(f"Input:  {to_relative(data_in)}")
print(f"Output: {to_relative(data_out)}")

GOOGLE_API_KEY present? True
Input:  Finetuning/data/Benchmark Data
Output: Finetuning/data


In [2]:
run_converter(data_in, data_out)

Scanning Finetuning/data/Benchmark Data...

--- Processing Training Set (43 Groups) ---
> room_booking_easy_1
  [Extraction] Querying Gemini for structure...
You are an expert in Answer Set Programming (specifically Clingo 5+ syntax). 
I will give you a full problem description (NL) and a full ASP solution.

Your task is to DECONSTRUCT this solution into the specific structural components required for a scheduler system.
Return the result as a valid JSON object.

### CODE REFACTORING & OPTIMIZATION (CRITICAL)
The input ASP code might be old, inefficient, or syntactically loose. Do NOT just copy-paste it.
**You have full authority to rewrite the logic** to ensure the highest quality training data:
1.  **Modernize:** Use modern Clingo 5+ syntax (e.g., `#count`, `#sum`). Replace deprecated constructs.
2.  **Optimize:** If a constraint is written inefficiently (e.g., using unnecessary helper predicates for simple checks), refactor it to be concise and readable.
3.  **Correctness:** If the 

: 

In [ ]:
train_file = os.path.join(data_out, "train.jsonl")

# 1. Load all lines into memory (efficient enough for small-medium datasets)
with open(train_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# 2. Pick a random line
if lines:
    random_line = random.choice(lines)
    sample = json.loads(random_line)

    print(f"--- RANDOM SAMPLE (Total Examples: {len(lines)}) ---")
    
    # 3. Iterate through messages and print nicely
    for msg in sample['messages']:
        role = msg['role'].upper()
        content = msg['content']
        
        print(f"\n[{role}]:")
        print("-" * 20)
        print(content)
        print("=" * 60)
else:
    print("Dataset file is empty.")

--- RANDOM SAMPLE (Total Examples: 387) ---

[SYSTEM]:
--------------------
You are an expert in Answer Set Programming. Task: Convert natural language instance descriptions into ASP facts and domain definitions. Output: Provide ONLY the valid Clingo code. Do not provide any explanation, comments, or markdown formatting.

[USER]:
--------------------
Problem Description:
A drone is deployed from headquarters at point 1 to scan a couple of points on a short
path network. There are four locations in total. Connections exist such that 1 links to 2
and 3; 2 links to 4. The drone is only allowed to fly between adjacent locations.
It has exactly 5 time units to complete its path, with each move
taking 1 unit of time. The scanning task requires it to visit locations 3 and 4, in any 
order. It begins at 1.

### TASK
Create the instance template (define predicates and domains).

[ASSISTANT]:
--------------------
loc(1..4). edge(1, 2). edge(1, 3). edge(2, 4). time(0..5).
